In [1]:
! git clone https://github.com/5amessi/license_plates.git

Cloning into 'license_plates'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 141 (delta 1), reused 0 (delta 0), pack-reused 135
Receiving objects: 100% (141/141), 24.37 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
! git clone https://github.com/SeyedHamidreza/car_plate_dataset.git


Cloning into 'car_plate_dataset'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48
Unpacking objects: 100% (48/48), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/car_plate_dataset/IRCP_dataset_1024X768.zip", 'r')
zip_ref.extractall("")
zip_ref.close()

In [5]:
import scipy
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import *
from keras.layers import *


class DataLoader():
    def __init__(self, dataset_name, img_res=(384, 192),out_res=(384, 192)):
        self.dataset_name = dataset_name
        self.img_res = img_res
        self.out_res = out_res

    def load_data(self,png = False):
        if png == True:
          path = glob('/content/%s/*.png' % (self.dataset_name))
        else:
          path = glob('/content/%s/*' % (self.dataset_name))
        
        imgs_hr = []
        imgs_lr = []

        
        for idx , i in enumerate(path):
          if idx >= 1000:
            break
          img = cv2.imread(i)
          w, h = self.img_res
          low_w, low_h = int(w / 3), int(h / 3)

          img_hr = cv2.resize(img, self.out_res)
          
          img_lr = cv2.resize(img, (low_w, low_h))
          img_lr = cv2.resize(img_lr, self.img_res)

          flr=np.fliplr(img_lr)
          fhr=np.fliplr(img_hr)

          imgs_hr.append(img_hr)
          imgs_hr.append(fhr)
          imgs_lr.append(img_lr)
          imgs_lr.append(flr)    

        imgs_hr = np.array(imgs_hr) / 127.5 - 1.
        imgs_lr = np.array(imgs_lr) / 127.5 - 1.

        return imgs_hr, imgs_lr

dl = DataLoader("license_plates")
hr , lr = dl.load_data()
print(np.shape(hr))
print(np.shape(lr))

dl = DataLoader("IRCP_dataset_1024X768")
hr2 , lr2 = dl.load_data()

print(np.shape(hr2))
print(np.shape(lr2))
hr = np.concatenate((hr, hr2))
lr = np.concatenate((lr, lr2))
hr2 = []
lr2 = []
print(np.shape(hr))
print(np.shape(lr))
t1 = (hr[0] + 1) * 127.5
t1 = np.array(t1, dtype=np.uint8)
cv2.imwrite('hr.jpg',t1)
t1 = (lr[0] + 1) * 127.5
t1 = np.array(t1, dtype=np.uint8)
cv2.imwrite('lr.jpg',t1)


Using TensorFlow backend.


(220, 192, 384, 3)
(220, 192, 384, 3)
(440, 192, 384, 3)
(440, 192, 384, 3)
(660, 192, 384, 3)
(660, 192, 384, 3)


True

In [0]:
def load_image_test(img):
  imgs_lr = []
  w, h = 384, 192

  img_lr = cv2.resize(img, (w,h))
  
  imgs_lr.append(img_lr)
  imgs_lr = np.array(imgs_lr) / 127.5 - 1.

  return imgs_lr

In [0]:
from keras.engine.saving import load_model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

class SRGAN():
    def __init__(self):
        self.channels = 3
        self.lr_height = 192                 
        self.lr_width = 384                 
        self.lr_shape = (self.lr_height, self.lr_width, self.channels)
        self.hr_height = self.lr_height  
        self.hr_width = self.lr_width    
        self.hr_shape = (self.hr_height, self.hr_width, self.channels)

        # Number of residual blocks in the generator
        self.n_residual_blocks = 16

        optimizer = Adam(0.0001)
        """
        pre-trained VGG19 model to extract image features from the high resolution image 
        and the generated high resolution images and minimize the mse between them
        """
        self.vgg = self.build_vgg()
        self.vgg.trainable = False
        self.vgg.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        patch = int(self.hr_height / 2**4)
        self.disc_patch = (patch, patch*2, 1)

        # Number of filters in the first layer of Gen and Disc
        self.gf = 64
        self.df = 32

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        self.generator = self.build_generator()

        img_hr = Input(shape=self.hr_shape)
        img_lr = Input(shape=self.lr_shape)

        fake_hr = self.generator(img_lr)

        fake_features = self.vgg(fake_hr)

        self.discriminator.trainable = False

        validity = self.discriminator(fake_hr)

        self.combined = Model([img_lr, img_hr], [validity, fake_features] )
        self.combined.compile(loss=['binary_crossentropy', 'mse'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)

    def build_vgg(self):

        vgg = VGG19(weights="imagenet")
        vgg.outputs = [vgg.layers[9].output]

        img = Input(shape=self.hr_shape)

        img_features = vgg(img)

        return Model(img, img_features)

    def build_generator(self):

        def residual_block(layer_input, filters):
            d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(layer_input)
            d = BatchNormalization(momentum=0.5)(d)
            d = Activation('relu')(d)
            d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
            d = BatchNormalization(momentum=0.5)(d)
            d = Add()([d, layer_input])
            return d

        img_lr = Input(shape=self.lr_shape)

        c1 = Conv2D(self.gf, kernel_size=9, strides=1, padding='same')(img_lr)
        c1 = Activation('relu')(c1)

        r = residual_block(c1, self.gf)
        for _ in range(self.n_residual_blocks - 1):
            r = residual_block(r, self.gf)

        c2 = Conv2D(self.gf, kernel_size=3, strides=1, padding='same')(r)
        c2 = BatchNormalization(momentum=0.5)(c2)
        c2 = Add()([c2, c1])

        gen_hr = Conv2D(self.channels, kernel_size=9, strides=1, padding='same', activation='tanh')(c2)

        return Model(img_lr, gen_hr)

    def build_discriminator(self):

        def d_block(layer_input, filters, strides=1, bn=True):
            d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.5)(d)
            return d

        d0 = Input(shape=self.hr_shape)

        d1 = d_block(d0, self.df, bn=False)
        d1 = d_block(d1, self.df, strides=2)
        d1 = d_block(d1, self.df*2)
        d1 = d_block(d1, self.df*2, strides=2)
        d1 = d_block(d1, self.df*4)
        d1 = d_block(d1, self.df*4, strides=2)
        d1 = d_block(d1, self.df*8)
        d1 = d_block(d1, self.df*8, strides=2)

        d1 = Dense(self.df*8)(d1)
       
        d1 = LeakyReLU(alpha=0.2)(d1)
        validity = Dense(1, activation='sigmoid')(d1)

        return Model(d0, validity)
    def pred(self ,count = 0,idx = 0):
        test = cv2.imread("/content/test.jpg")
        test = load_image_test(test)
        result = self.generator.predict(test)
        result = (result + 1) * 127.5
        result = np.array(result, dtype=np.uint8)
        cv2.imwrite("testout%d.jpg"%(count),result[0])

        result = self.generator.predict([[lr[idx]]])
        result = (result + 1) * 127.5
        result = np.array(result, dtype=np.uint8)
        cv2.imwrite("output%d.jpg"%(count),result[0])

        result = (lr[idx] + 1) * 127.5
        result = np.array(result, dtype=np.uint8)
        cv2.imwrite("input%d.jpg"%(count),result)

    def train(self, gw, dw, gepochs, depochs, batch_size =4,saved=False):
        
        if saved == True:
          self.generator.load_weights(gw)
          self.discriminator.load_weights(dw)

        #  Train Discriminator
        fake_hr = self.generator.predict(lr)

        valid = np.ones((np.shape(hr)[0],) + self.disc_patch)
        fake = np.zeros((np.shape(hr)[0],) + self.disc_patch)
        
        print("Train the discriminators")
        # Train the discriminators (original images = real / generated = Fake)
        self.discriminator.fit(hr, valid,verbose=1,batch_size=batch_size,epochs=depochs)
        self.discriminator.fit(fake_hr, fake,verbose=1,batch_size=batch_size,epochs=depochs*2)

        #  Train Generator

        # Extract features from hr image using pre-trained VGG19 model
        image_features = self.vgg.predict(hr)

        print("Train the generators")
        g_loss = self.combined.fit([lr, hr], [valid, image_features],verbose=1,batch_size=batch_size,epochs=gepochs)
        self.generator.save_weights("gdrive/My Drive/Colab Notebooks/generator.h5")
        self.discriminator.save_weights("gdrive/My Drive/Colab Notebooks/discriminator.h5")

In [0]:
gan = SRGAN()
gan.train(_,_,gepochs=50,depochs=10,batch_size=4)
gan.pred(0)


In [8]:
gan = SRGAN()

Instructions for updating:
Colocations handled automatically by placer.
574717952/574710816 [==============================] - 12s 0us/step


In [0]:
gw = "/content/gdrive/My Drive/Colab Notebooks/generator.h5"
dw = "/content/gdrive/My Drive/Colab Notebooks/discriminator.h5"

In [0]:
gan.train(gw,dw,gepochs=50,depochs=5,batch_size=4,saved=True)
gan.pred(1)


In [0]:
gan.train(gw,dw,gepochs=50,depochs=20,batch_size=4,saved=True)
gan.pred(2)


In [0]:
gan.train(gw,dw,gepochs=10,depochs=10,batch_size=4,saved=True)
gan.pred(3)


In [0]:
gan.train(gw,dw,gepochs=50,depochs=10,batch_size=4,saved=True)
gan.pred(4)

Train the discriminators
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/10
660/660 [==============================] - 9s 14ms/step - loss: 0.2352 - acc: 0.7552
Epoch 2/10
660/660 [==============================] - 6s 9ms/step - loss: 0.0031 - acc: 0.9974
Epoch 3/10
660/660 [==============================] - 6s 9ms/step - loss: 1.7764e-04 - acc: 0.9999
Epoch 4/10
660/660 [==============================] - 6s 9ms/step - loss: 9.8987e-05 - acc: 0.9999
Epoch 5/10
660/660 [==============================] - 6s 9ms/step - loss: 3.4023e-05 - acc: 1.0000
Epoch 6/10
660/660 [==============================] - 6s 9ms/step - loss: 3.9067e-05 - acc: 1.0000
Epoch 7/10
660/660 [==============================] - 6s 9ms/step - loss: 2.0625e-05 - acc: 1.0000
Epoch 8/10
660/660 [==============================] - 6s 9ms/step - loss: 1.4104e-05 - acc: 1.0000
Epoch 9/10
660/660 [==============================] - 6s 9ms/step - loss: 1.4802e-05 - acc: 1.0000
Epoch 10/10
660/660 [==============================] - 6s 9ms/step - loss: 1.3188e-05 - acc: 1.0000
Epoch 1/20
660/6